In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle


In [252]:
#utils

def carregar_pickle(nome_arquivo):
  with open(nome_arquivo, 'rb') as arquivo:
    objeto = pickle.load(arquivo)
  return objeto

In [253]:
raw_data_set = carregar_pickle('dataset_prom_16b_6w.pkl')
input_full = raw_data_set['inputs']
output_full = raw_data_set['outputs']
sequences_full = raw_data_set['sequences']

In [201]:
input_full.shape

(106, 17, 32)

In [254]:
input_train, input_test, output_train, output_test, sequence_train, sequence_test = train_test_split(input_full, output_full, 
sequences_full, train_size=0.75 , shuffle= True, random_state = 0);

In [186]:
input_train.shape

(79, 17, 32)

In [255]:
#utils

# binariza resultado da classificação da RN. 
#param: array -> Rn output, corte -> x > corte significa 1
def myClassify(array, corte):
    classifiedArray = []
    for x in array:
        if(x > corte):
            classifiedArray.append(1)
        else:
            classifiedArray.append(0)
    
    return classifiedArray

# Retorna lista com Ok/Nok para predição binarizada  e valor real
# param: arrPredict -> predict da rede
# param: arrReal -> valores reais de referência
def comparePredictOkNok(arrPredict, arrReal):

    sizePredict = len(arrPredict)
    sizeArrReal = len(arrReal)

    if(sizePredict != sizeArrReal):
        print('tamanho dos arrays é imcompatível')
        return

    size = sizeArrReal

    arrOkNok = []
    for i in range(0, size):
        if(arrPredict[i] == arrReal[i]):
            arrOkNok.append('OK')
        else:
            arrOkNok.append('NOK')
    
    return arrOkNok



# Retorna lista com Ok/Nok para predição binarizada  e valor real
# param: arrPredict -> predict da rede
# param: arrReal -> valores reais de referência
# param: sequenceTest -> sequencia correspondente
def comparePredictOkNokWithSeq(arrPredict, arrReal, sequenceTest):

    sizePredict = len(arrPredict)
    sizeArrReal = len(arrReal)

    if(sizePredict != sizeArrReal):
        print('tamanho dos arrays é imcompatível')
        return

    size = sizeArrReal

    arrOkNok = []
    for i in range(0, size):
        if(arrPredict[i] == arrReal[i]):
            arrOkNok.append('OK' + ' : ' + sequenceTest[i])
        else:
            arrOkNok.append('NOK' + ' : ' + sequenceTest[i])
    
    return arrOkNok
        


In [188]:
input_train[0].shape

(17, 32)

In [257]:
CNN = keras.Sequential()
CNN.add(keras.layers.Conv1D(10, kernel_size= 1, strides= 1, padding='valid', 
                            activation='relu', input_shape = (17, 32), use_bias= True))
#CNN.add(keras.layers.AveragePooling1D(pool_size=2, strides= 1, padding='valid'))
CNN.add(keras.layers.Flatten())
CNN.add(keras.layers.Dense(256, activation='relu'))
CNN.add(keras.layers.Dense(64, activation='relu'))
CNN.add(keras.layers.Dense(10, activation='relu'))
CNN.add(keras.layers.Dense(1, activation='sigmoid'))



In [190]:
CNN.summary();
keras.utils.plot_model(CNN);

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 17, 10)            330       
                                                                 
 flatten_12 (Flatten)        (None, 170)               0         
                                                                 
 dense_48 (Dense)            (None, 256)               43776     
                                                                 
 dense_49 (Dense)            (None, 64)                16448     
                                                                 
 dense_50 (Dense)            (None, 10)                650       
                                                                 
 dense_51 (Dense)            (None, 1)                 11        
                                                                 
Total params: 61,215
Trainable params: 61,215
Non-tra

In [258]:
CNN.compile(loss=keras.losses.BinaryCrossentropy(reduction='sum_over_batch_size'), metrics=['accuracy'])

In [259]:
CNN.fit(input_train, output_train, epochs= 20, batch_size = 1) # validation_split = 0.33

Epoch 1/20
79/79 [==============================] - 2s 6ms/step - loss: 0.7452 - accuracy: 0.5190
Epoch 2/20
79/79 [==============================] - 1s 6ms/step - loss: 0.5973 - accuracy: 0.6962
Epoch 3/20
79/79 [==============================] - 1s 10ms/step - loss: 0.3251 - accuracy: 0.8734
Epoch 4/20
79/79 [==============================] - 1s 7ms/step - loss: 0.1989 - accuracy: 0.8987
Epoch 5/20
79/79 [==============================] - 1s 8ms/step - loss: 0.1500 - accuracy: 0.9241
Epoch 6/20
79/79 [==============================] - 1s 9ms/step - loss: 0.0631 - accuracy: 0.9620
Epoch 7/20
79/79 [==============================] - 1s 8ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 8/20
79/79 [==============================] - 1s 18ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 9/20
79/79 [==============================] - 1s 9ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/20
79/79 [==============================] - 1s 7ms/step - loss: 8.6311e-04 - accuracy: 1.0000
Epoch 11/20
7

In [260]:

CNN.evaluate(input_test, output_test, batch_size= 1)





27/27 [==============================] - 0s 4ms/step - loss: 0.6125 - accuracy: 0.9630


[0.6125094890594482, 0.9629629850387573]

In [261]:
predictOnTest = CNN.predict(input_test)
predictOnTest

1/1 [==============================] - 0s 167ms/step


array([[1.86039415e-08],
       [1.00000000e+00],
       [3.40509176e-10],
       [9.99999464e-01],
       [1.00000000e+00],
       [1.13712427e-04],
       [1.08816810e-11],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.24576308e-10],
       [4.36667161e-11],
       [1.00000000e+00],
       [9.99924242e-01],
       [9.75806937e-02],
       [1.22266993e-05],
       [1.11261215e-07],
       [1.00000000e+00],
       [3.80937809e-10],
       [9.49474052e-03],
       [3.38625580e-01],
       [8.47268084e-14],
       [1.00000000e+00],
       [9.99766231e-01],
       [1.34568010e-08],
       [1.00000000e+00],
       [9.99998987e-01],
       [2.81040935e-04]], dtype=float32)

In [262]:
output_test

array([0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0])

In [263]:
classifiedArrayPredict = myClassify(predictOnTest, 0.5)
np.array(classifiedArrayPredict)

array([0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0])

In [264]:

comparePredictOkNokWithSeq(classifiedArrayPredict, output_test, sequence_test)

['OK : agagggtgtactccaagaagaggaagatgaggctagacgtctctgcatggagtatga',
 'OK : gcaaaaataaatgcttgactctgtagcgggaaggcgtattatgcacaccccgcgccg',
 'OK : ttcgtctccgcgactacgatgagatgcctgagtgcttccgttactggattgtcacca',
 'OK : gtactagagaactagtgcattagcttatttttttgttatcatgctaaccacccggcg',
 'OK : aaattaaaattttattgacttaggtcactaaatactttaaccaatataggcatagcg',
 'OK : tatgaccgaacgagtcaatcagaccgctttgactctggtattactgtgaacattatt',
 'OK : taacattaataaataaggaggctctaatggcactcattagccaatcaatcaagaact',
 'OK : tttctacaaaacacttgatactgtatgagcatacagtataattgcttcaacagaaca',
 'OK : atgcatttttccgcttgtcttcctgagccgactccctataatgcgcctccatcgaca',
 'OK : tattggcttgctcaagcatgaactcaaggctgatacggcgagacttgcgagccttgt',
 'OK : aacgagtcaatcagaccgctttgactctggtattactgtgaacattattcgtctccg',
 'OK : tctgaaatgagctgttgacaattaatcatcgaactagttaactagtacgcaagttca',
 'OK : cactaatttattccatgtcacacttttcgcatctttgttatgctatggttatttcat',
 'OK : gaggtggctatgtgtatgaccgaacgagtcaatcagaccgctttgactctggtatta',
 'OK : catgtcagcctcgacaacttgcataaatgctttcttgtagacgtgccctacgcgc

In [94]:
CNN.layers[0].get_weights()[0][0]

array([[ 0.23533559,  0.42990232, -0.25082487, -0.15267515,  0.26705414,
         0.41497964,  0.27988416,  0.37338895, -0.29958403, -0.04501176],
       [ 0.42684212,  0.07528386,  0.28260273, -0.10901958, -0.15239488,
         0.44041243,  0.01431945,  0.2730652 ,  0.4608476 ,  0.46495697],
       [ 0.45438316,  0.02100177,  0.29724106, -0.24716143, -0.4676595 ,
         0.4724816 , -0.3817394 ,  0.19320472,  0.42271256, -0.05679994],
       [ 0.15253878, -0.10971248, -0.04070297, -0.21874414, -0.08176634,
         0.28545243, -0.43704703,  0.18602677,  0.22525644,  0.47238237],
       [-0.46574974, -0.4418646 , -0.38255867, -0.34078255, -0.23755996,
         0.2713209 ,  0.27443305, -0.03233695,  0.24816659, -0.28621373],
       [ 0.07085161,  0.39532802, -0.03007789, -0.32714292,  0.49284223,
         0.18271512, -0.03359003, -0.21458982, -0.42486927,  0.06511235],
       [ 0.2713358 , -0.27115414,  0.34532204, -0.23495436, -0.43641958,
        -0.3368862 ,  0.12960823,  0.10995054